In [1]:
# Create Spark Session
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("Factor of cores") \
    .master("local[*]") \
    .config("spark.executor.instances", "4") \
    .config("spark.executor.cores", "2") \
    .config("spark.executor.memory", "1G") \
    .config("spark.driver.memory", "4G") \
    .getOrCreate()
spark

In [3]:
# Example DataFrame 1
_data = [
    ["C101", "Akshay", 21, "22-10-2001"],
    ["C102", "Sivay", 20, "07-09-2000"],
    ["C103", "Aslam", 23, "04-05-1998"],
]
_cols = ["ID", "NAME", "AGE", "DOB"]
df_1 = spark.createDataFrame(data = _data, schema = _cols)
df_1.printSchema()
df_1.show(10, False)

root
 |-- ID: string (nullable = true)
 |-- NAME: string (nullable = true)
 |-- AGE: long (nullable = true)
 |-- DOB: string (nullable = true)

+----+------+---+----------+
|ID  |NAME  |AGE|DOB       |
+----+------+---+----------+
|C101|Akshay|21 |22-10-2001|
|C102|Sivay |20 |07-09-2000|
|C103|Aslam |23 |04-05-1998|
+----+------+---+----------+



In [5]:
df_1.columns

['ID', 'NAME', 'AGE', 'DOB']

In [4]:
# Example DataFrame 2
_data = [
    ["C106", "Suku", "Indore", ["Maths", "English"]],
    ["C110", "Jack", "Mumbai", ["Maths", "English", "Science"]],
    ["C113", "Gopi", "Rajkot", ["Social Science"]],
]
_cols = ["ID", "NAME", "ADDRESS", "SUBJECTS"]
df_2 = spark.createDataFrame(data = _data, schema = _cols)
df_2.printSchema()
df_2.show(10, False)

root
 |-- ID: string (nullable = true)
 |-- NAME: string (nullable = true)
 |-- ADDRESS: string (nullable = true)
 |-- SUBJECTS: array (nullable = true)
 |    |-- element: string (containsNull = true)

+----+----+-------+-------------------------+
|ID  |NAME|ADDRESS|SUBJECTS                 |
+----+----+-------+-------------------------+
|C106|Suku|Indore |[Maths, English]         |
|C110|Jack|Mumbai |[Maths, English, Science]|
|C113|Gopi|Rajkot |[Social Science]         |
+----+----+-------+-------------------------+



In [6]:
# Now before we can merge the dataframes we have to add the extra columns from either dataframes
from pyspark.sql.functions import lit
# Lets add missing columns from df_2 to df_1
for col in df_2.columns:
    if col not in df_1.columns:
        df_1 = df_1.withColumn(col, lit(None))
        
# Lets add missing columns from df_1 to df_2
for col in df_1.columns:
    if col not in df_2.columns:
        df_2 = df_2.withColumn(col, lit(None))
        
# View the dataframes
df_1.show()
df_2.show()

+----+------+---+----------+-------+--------+
|  ID|  NAME|AGE|       DOB|ADDRESS|SUBJECTS|
+----+------+---+----------+-------+--------+
|C101|Akshay| 21|22-10-2001|   NULL|    NULL|
|C102| Sivay| 20|07-09-2000|   NULL|    NULL|
|C103| Aslam| 23|04-05-1998|   NULL|    NULL|
+----+------+---+----------+-------+--------+

+----+----+-------+--------------------+----+----+
|  ID|NAME|ADDRESS|            SUBJECTS| AGE| DOB|
+----+----+-------+--------------------+----+----+
|C106|Suku| Indore|    [Maths, English]|NULL|NULL|
|C110|Jack| Mumbai|[Maths, English, ...|NULL|NULL|
|C113|Gopi| Rajkot|    [Social Science]|NULL|NULL|
+----+----+-------+--------------------+----+----+



In [7]:
# Lets use unionByName to do the merge successfully
df = df_1.unionByName(df_2)
df.printSchema()
df.show(10, False)

root
 |-- ID: string (nullable = true)
 |-- NAME: string (nullable = true)
 |-- AGE: long (nullable = true)
 |-- DOB: string (nullable = true)
 |-- ADDRESS: string (nullable = true)
 |-- SUBJECTS: array (nullable = true)
 |    |-- element: string (containsNull = true)

+----+------+----+----------+-------+-------------------------+
|ID  |NAME  |AGE |DOB       |ADDRESS|SUBJECTS                 |
+----+------+----+----------+-------+-------------------------+
|C101|Akshay|21  |22-10-2001|NULL   |NULL                     |
|C102|Sivay |20  |07-09-2000|NULL   |NULL                     |
|C103|Aslam |23  |04-05-1998|NULL   |NULL                     |
|C106|Suku  |NULL|NULL      |Indore |[Maths, English]         |
|C110|Jack  |NULL|NULL      |Mumbai |[Maths, English, Science]|
|C113|Gopi  |NULL|NULL      |Rajkot |[Social Science]         |
+----+------+----+----------+-------+-------------------------+



In [8]:
spark.stop()